In [4]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
import statsmodels.api as sm
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
from statsmodels.tsa.arima.model import ARIMA
from sklearn.cluster import KMeans


In [2]:
path = "d:\moje_dokumenty\Desktop\matfyz\mgr\semester 11\Projektovy seminar\dhi_data2"

In [6]:
files = os.listdir(path)

In [78]:
dataframes_dict = {}

# Import each file as a separate DataFrame and assign headers
for file_name in files:
    if file_name.endswith('.csv'):  # Check if the file is a CSV file
        # Extract headers from the filename by splitting it by "_"
        headers = file_name.split("_")
        header = "_".join(headers[1:])
        header = header.split(".")[0]
        headers = ["date", header]

        # Read the CSV file into a DataFrame, specifying column names as headers
        file_path = os.path.join(path, file_name)
        df = pd.read_csv(file_path, header=None, names=headers, sep=";")
        df["date"] = pd.to_datetime(df['date'], format='%d.%m.%Y %H:%M:%S')

        # Store the DataFrame in the dictionary with the filename as the key
        dataframes_dict[file_name] = df

In [79]:
sites = set(x.split("_")[0] for x in files)
site_dataframes = {}

for site in sites:
    df = pd.DataFrame(columns=['date'])
    for file in files:
        if file.startswith(site):
            site_df = dataframes_dict[file]
            df = pd.merge(df, site_df, on='date', how='outer')
    site_dataframes[site] = df

In [84]:
sites

{'MP1', 'MP2', 'MP3', 'MP4', 'MP5', 'MP6', 'MP7'}

In [80]:
i=-1

In [81]:
i+=1
site_dataframes[list(sites)[i]]

,date,hladina_result,hladina_TL_zdroj,hladina_UZ_zdroj,prutok_computed,rychlost_result,rychlost_zdroj
0,2022-06-15 09:55:00,0.07800,0.078,0.083,0.00522,0.184,0.184
1,2022-06-15 09:57:00,0.07800,0.078,0.082,0.00527,0.186,0.186
2,2022-06-15 09:59:00,0.07600,0.076,0.081,0.00505,0.185,0.185
3,2022-06-15 10:01:00,0.07600,0.076,0.081,0.00510,0.187,0.187
4,2022-06-15 10:03:00,0.07700,0.077,0.082,0.00523,0.188,0.188
...,...,...,...,...,...,...,...
66252,2022-09-15 10:19:00,0.07099,0.071,0.081,0.00459,0.186,0.186
66253,2022-09-15 10:21:00,0.06399,0.064,0.076,0.00353,0.167,0.167
66254,2022-09-15 10:23:00,NaN,0.070,0.074,NaN,0.164,0.164
66255,2022-09-15 10:25:00,NaN,0.017,0.079,NaN,0.230,0.230


In [87]:
for site in sorted(sites):
    print(site)
    df = site_dataframes[site]
    print(df.shape)
    print(site + " NAs:")
    print(df.isna().sum())
    print(f"{site} from {df['date'].min()} to {df['date'].max()}")
    print(site + " observation intervals:")
    print(df['date'].diff().dropna().value_counts())
    print(df["prutok_computed"].describe())

MP1
(99337, 7)
MP1 NAs:
date                    0
hladina_result      18865
hladina_TL_zdroj      371
hladina_UZ_zdroj      386
prutok_computed     18865
rychlost_result     18794
rychlost_zdroj        371
dtype: int64
MP1 from 2022-06-30 11:08:00 to 2022-10-20 08:16:00
MP1 observation intervals:
0 days 00:02:00        99290
0 days 00:01:00           34
0 days 00:04:00            3
0 days 00:03:00            1
0 days 00:06:00            1
0 days 00:12:00            1
-1 days +23:12:00          1
0 days 00:14:00            1
0 days 00:26:00            1
-112 days +03:34:00        1
-112 days +04:44:00        1
85 days 16:49:00           1
Name: date, dtype: int64
count    80472.000000
mean         0.184037
std          0.127910
min          0.052010
25%          0.125930
50%          0.163455
75%          0.196762
max          1.745810
Name: prutok_computed, dtype: float64
MP2
(66257, 7)
MP2 NAs:
date                   0
hladina_result         3
hladina_TL_zdroj    2092
hladina_UZ_zdroj